[ref](https://github.com/xiaopeng-liao/DEC_pytorch)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from sklearn.cluster import MiniBatchKMeans, KMeans

In [ ]:
from __future__ import print_function, division
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd

### Custom Dataloader for the GW Dataset

In [ ]:
class MyDataset(Dataset):

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.filelist = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.filelist.iloc[idx, 0])
        sample = io.imread(img_name)
        label = self.filelist.iloc[idx, 1]
        # landmarks = np.array([landmarks])
        # landmarks = landmarks.astype('float').reshape(-1, 2)
        # sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample, label

### Pretrain Model Data

In [ ]:
## UPPER Atmospheric Data
train_csv = '/content/drive/MyDrive/Seraj/upper-atmospheric-128/skybased-128-pretrain/pretrain_sky_train.csv'
test_csv = '/content/drive/MyDrive/Seraj/upper-atmospheric-128/skybased-128-pretrain/pretrain_sky_test.csv'
root_dir = '/content/drive/MyDrive/Seraj/upper-atmospheric-128/skybased-128-pretrain'

In [ ]:
batch_size = 10

train_set = MyDataset(train_csv,root_dir)
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)

test_set = MyDataset(test_csv, root_dir)
test_loader = torch.utils.data.DataLoader(
                 dataset=test_set,
                 batch_size=batch_size,
                 shuffle=True)

data_sample = test_loader.dataset[0]
x,y = data_sample
print(x.shape)
# print(len(test_loader))

torch.Size([1, 128, 128])


In [ ]:
import time
timingResult = {}
def logTime(theName, currentTime):
    if theName not in timingResult:
        timingResult[theName] = time.time() - currentTime
    else:
        timingResult[theName] = timingResult[theName] + (time.time() - currentTime)
    currentTime = time.time()
    return currentTime

def printTiming(name):
    print('======== timing for {}: {} ======='.format(name,timingResult[name]))

In [ ]:
class DEC_AE(nn.Module):
    def __init__(self, num_classes, num_features):
        super(DEC_AE,self).__init__()
        self.dropout = nn.Dropout(p=0.3)
        # self.dropout = nn.AlphaDropout(0.5)
        # self.fc1 = nn.Linear(1000*1000*4,500)
        # self.fc1 = nn.Linear(28*28*1,500)
        self.fc1 = nn.Linear(128*128*1,500)
        # self.fc1 = nn.Linear(512*512*1,500)
        self.fc2 = nn.Linear(500,500)
        self.fc3 = nn.Linear(500,2000)
        self.fc4 = nn.Linear(2000,num_features)
        self.relu = nn.ReLU()
        
        # self.fc_d1 = nn.Linear(500,1000*1000*4)
        # self.fc_d1 = nn.Linear(500,28*28*1)
        self.fc_d1 = nn.Linear(500,128*128*1)
        # self.fc_d1 = nn.Linear(500,512*512*1)
        self.fc_d2 = nn.Linear(500,500)
        self.fc_d3 = nn.Linear(2000,500)
        self.fc_d4 = nn.Linear(num_features,2000)
        self.alpha = 1.0
        self.clusterCenter = nn.Parameter(torch.zeros(num_classes,num_features))
        self.pretrainMode = True
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform(m.weight)

    def setPretrain(self,mode):
        """To set training mode to pretrain or not, 
        so that it can control to run only the Encoder or Encoder+Decoder"""
        self.pretrainMode = mode
    def updateClusterCenter(self, cc):
        """
        To update the cluster center. This is a method for pre-train phase.
        When a center is being provided by kmeans, we need to update it so
        that it is available for further training
        :param cc: the cluster centers to update, size of num_classes x num_features
        """
        self.clusterCenter.data = torch.from_numpy(cc)
    def getTDistribution(self,x, clusterCenter):
        """
        student t-distribution, as same as used in t-SNE algorithm.
         q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
         
         :param x: input data, in this context it is encoder output
         :param clusterCenter: the cluster center from kmeans
         """
        xe = torch.unsqueeze(x,1).cuda() - clusterCenter.cuda()
        q = 1.0 / (1.0 + (torch.sum(torch.mul(xe,xe), 2) / self.alpha))
        q = q ** (self.alpha + 1.0) / 2.0
        q = (q.t() / torch.sum(q, 1)).t() #due to divison, we need to transpose q
        return q
        
    def forward(self,x):
        # x = x.view(-1, 1*28*28)
        x = x.view(-1,1*128*128)
        # x = x.view(-1,1*512*512)
        # x = x.view(-1, 4*1000*1000)
        # 32x32x1
        x = self.dropout(x)
        # 32x32x1
        x = self.fc1(x)
        # 17x17x50
        x = self.relu(x)
        # 17x17x50
        x = self.fc2(x)
        # 17x17x50
        x = self.relu(x)
        # 9x9x50
        x = self.fc3(x)
        # 17x17x50
        x = self.relu(x)
        x = self.fc4(x)
        # 9x9x50
        x_ae = x
        #if not in pretrain mode, we only need encoder
        if self.pretrainMode == False:
            return x, self.getTDistribution(x,self.clusterCenter)
        # 1x68
        ##### encoder is done, followed by decoder #####
        # 1x68
        x = self.fc_d4(x)
        # 1x4050
        x = self.relu(x)
        # 1x4050
        x = self.fc_d3(x)
        # 1x4050
        x = self.relu(x)
        x = self.fc_d2(x)
        # 1x4050
        x = self.relu(x)
        x = self.fc_d1(x)
        # x_de = x.view(-1,1,28,28)
        x_de = x.view(-1,1,128,128)
        # x_de = x.view(-1,1,512,512)
        # x_de = x.view(-1, 4*1000*1000)
        # x_de = torch.reshape(x,(-1,1,128,128)) ## for testing
        # 1x4050
        return x_ae, x_de

In [ ]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score

def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    # from sklearn.utils.linear_assignment_ import linear_assignment
    from scipy.optimize import linear_sum_assignment as linear_assignment
    ind = linear_assignment(w.max() - w)
    a, b =ind
    # print(w.shape)
    # print(ind)
    temp=0
    for i in range(len(a)):
      temp += np.sum(w[a[i], b[i]])
    # return np.sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size
    return (temp* 1.0 / y_pred.size)

In [ ]:
class DEC:
    """The class for controlling the training process of DEC"""
    def __init__(self,n_clusters,alpha=1.0):
        self.n_clusters=n_clusters
        self.alpha = alpha
        
    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        #print('q',q)
        return Variable((weight.t() / weight.sum(1)).t().data, requires_grad=True)
    def logAccuracy(self,pred,label):
        print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
          % (acc(label, pred), nmi(label, pred)))
    @staticmethod
    def kld(q,p):
        res = torch.sum(p*torch.log(p/q),dim=-1)
        # res = torch.sum(p*torch.log(p)-q*torch.log(q),dim=-1)
        return res
    
    def validateOnCompleteTestData(self,test_loader,model):
        model.eval()
        to_eval = np.array([model(d[0].cuda())[0].data.cpu().numpy() for i,d in enumerate(test_loader)])
        # print("++++++++++++++++++", to_eval.shape)
        true_labels = np.array([d[1].cpu().numpy() for i,d in enumerate(test_loader)])
        # print("++++++++++++++++++", true_labels.shape)
        to_eval = np.reshape(to_eval,(to_eval.shape[0]*to_eval.shape[1],to_eval.shape[2]))
        # to_eval = np.reshape(to_eval,(to_eval.shape[0]*to_eval.shape[1]))
        true_labels = np.reshape(true_labels,true_labels.shape[0]*true_labels.shape[1])
        km = KMeans(n_clusters=len(np.unique(true_labels)), n_init=20)
        y_pred = km.fit_predict(to_eval)
        print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                      % (acc(true_labels, y_pred), nmi(true_labels, y_pred)))
        currentAcc = acc(true_labels, y_pred)
        return currentAcc
    
    def pretrain(self,train_loader, test_loader, epochs):
        # dec_ae = DEC_AE(10,10).cuda() #auto encoder
        dec_ae = DEC_AE(2,10).cuda()
        mseloss = nn.MSELoss()
        optimizer = optim.SGD(dec_ae.parameters(),lr = 0.0001, momentum=0.9)
        best_acc = 0.0
        for epoch in range(epochs):
            dec_ae.train()
            running_loss=0.0
            for i,data in enumerate(train_loader):
                # print("I am here")
                x, label = data
                x, label = Variable(x).cuda(),Variable(label).cuda()
                optimizer.zero_grad()
                x_ae,x_de = dec_ae(x)
                # x = np.reshape(x,(batch_size,-1))
                # x =torch.reshape(x,(batch_size,128*128))
                # print(x.shape)
                loss = F.mse_loss(x_de,x,reduce=True) 
                # loss = mseloss(x_de, x)
                loss.backward()
                optimizer.step()
                x_eval = x.data.cpu().numpy()
                label_eval = label.data.cpu().numpy()
                running_loss += loss.data.cpu().numpy()
                if i % 10 == 9:    # print every 10 mini-batches
                    print('[%d, %5d] loss: %.7f' %
                          (epoch + 1, i + 1, running_loss / 10))
                    running_loss = 0.0
            #now we evaluate the accuracy with AE
            dec_ae.eval()
            currentAcc = self.validateOnCompleteTestData(test_loader,dec_ae)
            if currentAcc > best_acc:                
                torch.save(dec_ae,'pretrain_EnDe.pth'.format(best_acc))
                best_acc = currentAcc
    def clustering(self,mbk,x,model):
        model.eval()
        y_pred_ae,_ = model(x)
        y_pred_ae = y_pred_ae.data.cpu().numpy()
        y_pred = mbk.partial_fit(y_pred_ae) #seems we can only get a centre from batch
        self.cluster_centers = mbk.cluster_centers_ #keep the cluster centers
        model.updateClusterCenter(self.cluster_centers)
    def train(self,train_loader, test_loader, epochs):
        """This method will start training for DEC cluster"""
        ct = time.time()
        model = torch.load("pretrain_EnDe.pth").cuda()
        model.setPretrain(False)
        optimizer = optim.SGD([{'params': model.parameters()}, ],lr = 0.0001, momentum=0.9)
        # optimizer = optim.Adam([{'params': model.parameters()}, ],lr = 0.1) #, momentum=0.9
        print('Initializing cluster center with pre-trained weights')
        mbk = MiniBatchKMeans(n_clusters=self.n_clusters, n_init=20, batch_size=batch_size)
        got_cluster_center = False
        for epoch in range(epochs):
            for i,data in enumerate(train_loader):
                x, label = data
                x = Variable(x).cuda()
                optimizer.zero_grad()
                #step 1 - get cluster center from batch
                #here we are using minibatch kmeans to be able to cope with larger dataset.
                if not got_cluster_center:
                    self.clustering(mbk,x,model)
                    if epoch > 1:
                        got_cluster_center = True
                else:
                    model.train()
                    #now we start training with acquired cluster center
                    feature_pred,q = model(x)
                    #get target distribution
                    p = self.target_distribution(q)
                    #print('q',q,'p',p)
                    loss = self.kld(q,p).mean()
                    loss.backward()
                    optimizer.step()
            currentAcc = self.validateOnCompleteTestData(test_loader,model)
        # return model

In [ ]:
#now start training
import random
# random.seed(7)
dec = DEC(2)
dec.pretrain(train_loader, test_loader, 15) ## parameter initialze

[1,    10] loss: 0.4577536
[1,    20] loss: 0.4961189
        |==>  acc: 0.5125,  nmi: 0.0005  <==|
[2,    10] loss: 0.4674316
[2,    20] loss: 0.5120804
        |==>  acc: 0.5375,  nmi: 0.0042  <==|
[3,    10] loss: 0.4531930
[3,    20] loss: 0.5124784
        |==>  acc: 0.5375,  nmi: 0.0042  <==|
[4,    10] loss: 0.4846529
[4,    20] loss: 0.4830212
        |==>  acc: 0.5375,  nmi: 0.0042  <==|
[5,    10] loss: 0.4722325
[5,    20] loss: 0.5013712
        |==>  acc: 0.5125,  nmi: 0.0005  <==|
[6,    10] loss: 0.4947464
[6,    20] loss: 0.4683196
        |==>  acc: 0.5625,  nmi: 0.0118  <==|
[7,    10] loss: 0.4753345
[7,    20] loss: 0.4840784
        |==>  acc: 0.5375,  nmi: 0.0042  <==|
[8,    10] loss: 0.5079423
[8,    20] loss: 0.4538487
        |==>  acc: 0.6125,  nmi: 0.0385  <==|
[9,    10] loss: 0.4574465
[9,    20] loss: 0.5082907
        |==>  acc: 0.6375,  nmi: 0.0578  <==|
[10,    10] loss: 0.4724823
[10,    20] loss: 0.4842722
        |==>  acc: 0.5375,  nmi: 0.0042  <==

In [ ]:
dec.train(train_loader, test_loader, 15) ## DEC train

Initializing cluster center with pre-trained weights
        |==>  acc: 0.5625,  nmi: 0.0118  <==|
        |==>  acc: 0.5375,  nmi: 0.0042  <==|
        |==>  acc: 0.5125,  nmi: 0.0005  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5500,  nmi: 0.0076  <==|
        |==>  acc: 0.5750,  nmi: 0.0172  <==|
        |==>  acc: 0.6000,  nmi: 0.0308  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5625,  nmi: 0.0118  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|
        |==>  acc: 0.5250,  nmi: 0.0019  <==|


### Encoder + Decoder Model

In [ ]:
class MyModel(nn.Module):
    def __init__(self, in_features, nb_classes, pretrained):
        super(MyModel, self).__init__()
        if nb_classes==2:
          self.nb_classes=1

        self.pretrained = pretrained
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32,out_channels=1,kernel_size=3, stride=2)
        self.relu = nn.ReLU() 
        self.final_fc = nn.Linear(225, self.nb_classes)
        self.flaten = nn.Flatten()
        # self.dropout = nn.Dropout2d(0.4)
        self.out = nn.Softmax()  # for hinge loss turn it off

    def forward(self, x):
        self.pretrained.eval()
        # x, _ = self.pretrained(x) # encoder output
        _, x = self.pretrained(x) # decoder output
        x = self.conv1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.relu(x)

        # x = self.conv1(x)
        # x = self.relu(x)

        # x = self.dropout(x)

        x = self.flaten(x)
        x   = self.final_fc(x) 
        x = self.out(x) # turn it off for hingeloss
        return x

In [ ]:
!pip install torchinfo
from torchinfo import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## load pretrained mdoel 
pretrained = torch.load("pretrain_EnDe.pth")
model = MyModel(10,2,pretrained)
summary(model, input_size=(10,1,128,128))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [10, 1]                   --
├─DEC_AE: 1-1                            [10, 10]                  20
│    └─Dropout: 2-1                      [10, 16384]               --
│    └─Linear: 2-2                       [10, 500]                 8,192,500
│    └─ReLU: 2-3                         [10, 500]                 --
│    └─Linear: 2-4                       [10, 500]                 250,500
│    └─ReLU: 2-5                         [10, 500]                 --
│    └─Linear: 2-6                       [10, 2000]                1,002,000
│    └─ReLU: 2-7                         [10, 2000]                --
│    └─Linear: 2-8                       [10, 10]                  20,010
│    └─Linear: 2-9                       [10, 2000]                22,000
│    └─ReLU: 2-10                        [10, 2000]                --
│    └─Linear: 2-11                      [10, 500]        

##### [suggested BCEWithLogitsLoss or CrossEntropyLoss](https://discuss.pytorch.org/t/model-accuracy-stuck-at-0-5-though-loss-is-consistently-decreasing/72343/4)

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
device ='cuda'
model = model.to(device)

criterion =  nn.BCEWithLogitsLoss() ## suggested in the above link
# criterion = nn.CrossEntropyLoss() ## suggested in the above link ## good for imbalanced dataset

# criterion =  nn.BCELoss()
# criterion =  nn.HingeEmbeddingLoss()


# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model.parameters(), lr=0.1) ##.1, .01, .001, .2

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
n_epochs = 20

In [ ]:
## UPPER Atmospheric Data
train_csv = '/content/drive/MyDrive/Seraj/skybased-128-downstream/downstream_sky_train.csv'
test_csv = '/content/drive/MyDrive/Seraj/skybased-128-downstream/downstream_sky_test.csv'
root_dir = '/content/drive/MyDrive/Seraj/skybased-128-downstream'

In [ ]:
batch_size = 10
train_set = MyDataset(train_csv,root_dir)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_set = MyDataset(test_csv, root_dir)
# test_set = MyDataset(test_csv,root_dir)

test_loader = torch.utils.data.DataLoader(
                 dataset=test_set,
                 batch_size=batch_size,
                 shuffle=False)

data_sample = train_loader.dataset[0]
x,y = data_sample
print(x.shape)

torch.Size([1, 128, 128])


In [ ]:
train_losses = []
train_acc = []

for epoch in range(n_epochs):
    model.train()
    running_loss=0.0
    train_epoch_loss = []
    train_epoch_acc = []
    for i,data in enumerate(train_loader):
      x, label = data
      x, label = Variable(x).cuda(),Variable(label).float().cuda()
      optimizer_ft.zero_grad()
      pred = model(x)
  
      label = torch.unsqueeze(label,1)
      loss = criterion(pred,label)
      loss.backward()
      optimizer_ft.step()
      
      y_true = label.cpu().numpy().astype(int)
      y_pred = pred.cpu().detach().numpy()
      y_pred = np.argmax(y_pred,axis=1)
      accuracy = acc(y_pred,y_true)

      train_epoch_acc.append(accuracy)
      train_epoch_loss.append(loss.data.cpu().numpy())

      print('[%d, %5d] loss: %.7f' % (epoch + 1, i + 1, loss.data.cpu().numpy() ))
      print('acc', np.average(train_epoch_acc), 'loss',np.average(train_epoch_loss))


[1,     1] loss: 0.8132618
[1,     2] loss: 1.0132618
[1,     3] loss: 0.9132618
[1,     4] loss: 0.8132618
[1,     5] loss: 0.8132618
[1,     6] loss: 0.5132617
[1,     7] loss: 0.8132618
[1,     8] loss: 0.7132618
[1,     9] loss: 1.2132618
[1,    10] loss: 0.7132618
[1,    11] loss: 0.7132618
[1,    12] loss: 1.0132618
[1,    13] loss: 0.7132618
[1,    14] loss: 0.9132618
[1,    15] loss: 0.8132618
[1,    16] loss: 0.7132618
[1,    17] loss: 0.6132618
[1,    18] loss: 1.0132618
[1,    19] loss: 0.8132618
[1,    20] loss: 0.6132618
[1,    21] loss: 0.9132618
[1,    22] loss: 0.7132618
acc 0.618181818181818 loss 0.8132618
[2,     1] loss: 0.8132618
[2,     2] loss: 0.9132618
[2,     3] loss: 0.9132618
[2,     4] loss: 0.7132618
[2,     5] loss: 0.9132618
[2,     6] loss: 0.9132618
[2,     7] loss: 0.6132618
[2,     8] loss: 0.7132618
[2,     9] loss: 0.8132618
[2,    10] loss: 0.8132618
[2,    11] loss: 0.6132618
[2,    12] loss: 1.0132618
[2,    13] loss: 0.7132618
[2,    14] loss: 1

In [ ]:
test_losses = []
test_acc = []

model.eval()

for i in range(len(test_loader)):
  data = test_loader.dataset[i]
  x, label = data
  label = torch.Tensor(label)

  x, label = Variable(x).cuda(),Variable(label).float().cuda()
  optimizer_ft.zero_grad()
  pred = model(x)
  
  label = torch.unsqueeze(label,1)
  loss = criterion(pred,label)
  
  # Acc
  y_true = label.cpu().numpy().astype(int)
  y_pred = pred.cpu().detach().numpy()
  y_pred = np.argmax(y_pred,axis=1)
  print(y_pred.shape)
  print('newwww', y_true.shape)
  accuracy = acc(y_pred,y_true)

  test_acc.append(accuracy)

  test_losses.append(loss.data.cpu().numpy())

  print('[%5d] loss: %.7f' % (i + 1, loss.data.cpu().numpy() ))
  print('[%5d] acc: %.7f' % (i + 1, accuracy ))

(1,)
newwww (1, 1)
[    1] loss: 1.3132617
[    1] acc: 1.0000000
(1,)
newwww (1, 1)
[    2] loss: 1.3132617
[    2] acc: 1.0000000
(1,)
newwww (1, 1)
[    3] loss: 1.3132617
[    3] acc: 1.0000000
(1,)
newwww (1, 1)
[    4] loss: 1.3132617
[    4] acc: 1.0000000
(1,)
newwww (1, 1)
[    5] loss: 1.3132617
[    5] acc: 1.0000000
(1,)
newwww (1, 1)
[    6] loss: 1.3132617
[    6] acc: 1.0000000
(1,)
newwww (1, 1)
[    7] loss: 1.3132617
[    7] acc: 1.0000000
(1,)
newwww (1, 1)
[    8] loss: 1.3132617
[    8] acc: 1.0000000
